In [143]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
import pickle
import re

In [144]:
url = 'https://www.shapeyourcity.ca/development'
PATH = "C:\Program Files (x86)\chromedriver-win64\chromedriver.exe"

In [145]:
def get_list_of_urls(webpage,path):
    """
    Scrapes all the URLs of the development permits within the webpage

    Parameters:
        webpage (str): the url of the webpage
        path (str): the location of the user's chromedriver

    Returns:
        A list of strings which contains all of the development permits url 
    """
    
    chrome_options = Options()
    chrome_options.add_argument("--headless=new")
    driver = webdriver.Chrome(service = Service(path), options=chrome_options)
    driver.get(webpage)
    # The element is located within an iframe, required to locate the iframe and switch frames
    iframe = driver.find_element(By.TAG_NAME, 'iframe')
    url_list = []
    driver.switch_to.frame(iframe)
    page_num = WebDriverWait(driver, 20).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, ".chakra-button.ehq-paginationButton.css-i1louw")))
    last_page_num = int([item.text for item in page_num][-2])
    # Scrape all of the urls on each page
    for num in range(last_page_num+1):
        # Ensures that all the CSS elements are loaded before scraping
        urls = WebDriverWait(driver, 20).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, '.chakra-link.ehq-projectCoverImg.css-1eh7kaa')))
        for url in urls:
            url_list.append(url.get_attribute('href'))
        # After scraping all of the elements, click to the next page if not on the final page
        if num < last_page_num+1:
            click = driver.find_element(By.CSS_SELECTOR, ".chakra-button.ehq-paginationButton.ehq-paginationNextButton.css-i1louw")
            click.click()
        else:
            break
        time.sleep(5)
    driver.switch_to.default_content()
    driver.quit()
    return url_list

In [78]:
all_permit_urls = get_list_of_urls(url, PATH)
# save the urls to a pickle file that can be opened at any time without re-running the code. 
with open('permit_urls', 'wb') as f:
    pickle.dump(all_permit_urls, f)

In [4]:
# Load the saved pickle with all the URLs.
all_permit_urls_saved = pd.read_pickle('permit_urls')

In [146]:
def get_list_of_applicant(list_of_description):
    """
    Utilizes regex to obtain the applicants name from the list of description

    Parameters:
        list_of_description (list of str): the description of the webpage derived from the get_description 

    Returns:
        A list of the applicant names 
    """
    
    regex = r'^[\s\S]*?(?=\s+has applied)'
    list_of_applicant = []
    for applicant in list_of_description:
        try:
            applicant_name = applicant[:re.search(regex, applicant).span()[1]]
        except:
            applicant_name = 'Unknown'
        list_of_applicant.append(applicant_name)
    return list_of_applicant

In [147]:
def application_status(driver):
    """
    Scrape webpage to obtain the Director of Planning decision

    Parameters:
        driver: the driver that opened the webpage

    Returns:
        The application status
    """  

    try:
        text_info = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.TAG_NAME,'strong'))).text.split()
        if 'Director' in text_info:
            if 'approved' in text_info:
                return 'Approved'
            elif 'cancelled' in text_info:
                return 'Cancelled'
            elif 'withdrawn' in text_info:
                return 'Withdrawn'
            else:
                return 'Rejected'
        else:
            return 'In progress'
    except:
        return 'In progress'

In [148]:
def permit_status(driver, regex):
    """
    Scrapes webpage to obtain the development permit id

    Parameters:
        driver: the driver that opened the webpage
        regex: the regular expression required to isolate the permit id

    Returns:
        The development permit id 
    """
    
    try:
        head_text = driver.find_element(By.TAG_NAME, 'h1').text
        permit_id = re.search(regex, head_text)[1] 
    except:
        permit_id = 'Unknown'
    return permit_id

In [149]:
def scrape_description(driver):
    """
    Scrapes each webpage to obtain the description of each development permit application

    Parameters:
        driver: the driver that opened the webpage

    Returns:
        The description of the development permit application 
    """

    text = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME, 'truncated-description'))).text
    return text

In [150]:
def scrape_info(list_of_urls, path):
    """
    Scrapes each webpage in the list of urls

    Parameters:
        list_of_urls (list of str): all the urls to be scraped
        path (str): the location of the user's chromedriver

    Returns:
        A tuple where the 0th element is the list status, the 1st element is the description list, and the 2nd element is the permit list
    """

    chrome_options = Options()
    chrome_options.add_argument("--headless=new")
    list_status = []
    description_list = []
    permit_list = []
    error_list = []
    regex_permit = r'\((DP.*?)\)'
    for url in list_of_urls:
        driver = webdriver.Chrome(service = Service(path), options = chrome_options)
        driver.get(url)
        try:
            list_status.append(application_status(driver))
            permit_list.append(permit_status(driver, regex_permit))
            description_list.append(scrape_description(driver))
            driver.quit()
            time.sleep(3)
        except:
            error_list.append(url)
            driver.quit()
            time.sleep(3)
    return list_status, description_list, permit_list, error_list


In [151]:
all_information = scrape_info(all_permit_urls_saved, PATH)

In [152]:
# Save all the scraped information
with open('scraped_data', 'wb') as f:
    pickle.dump(all_information, f)

In [153]:
# Get Applicant name from the description
applicant = get_list_of_applicant(all_information[1])

In [154]:
# Create the dataframe to hold all the necessary information
df = pd.DataFrame({"Applicant_Name": applicant})
df['permit_id'] = all_information[2]
df['applicant_status'] = all_information[0]
df['description'] = all_information[1]

In [155]:
# save the database
with open('database', 'wb') as f:
    pickle.dump(df, f)

In [160]:
df[df['Applicant_Name'] == 'Unknown']

,Applicant_Name,permit_id,applicant_status,description
0,Unknown,DP-2023-00821,In progress,The City of Vancouver has received an applicat...
40,Unknown,DP-2023-00289,Approved,The City of Vancouver has received a request f...
47,Unknown,DP-2023-00556,Approved,The City of Vancouver has received an applicat...
48,Unknown,DP-2023-00580,Approved,The City of Vancouver has received a request f...
68,Unknown,DP-2023-00719,In progress,The City of Vancouver has received a request f...
84,Unknown,DP-2023-00711,Approved,Listraor & Locarno have applied to the City of...
94,Unknown,DP-2019-00058,In progress,The City of Vancouver has received an applicat...
107,Unknown,DP-2022-00858,Approved,The City of Vancouver has received a request f...
118,Unknown,DP-2023-00327,In progress,Wall Financial Corporation has submitted a DP ...
121,Unknown,DP-2023-00596,In progress,Gracorp Properties LP has submitted a DP appli...


In [ ]:
# Grab first item of current url list, scrape first page of website, check each url of the first page of website until it matches the first item, stop loop, collect urls in a list and extend this list with the old one. 